In [ ]:
# Imports
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

# Initialize the environment
env = gym.make('LunarLander-v3')
env.reset() # Reset to get observation space and action space details

print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")
print(f"Sample observation: {env.observation_space.sample()}")
print(f"Sample action: {env.action_space.sample()}")

# Hyperparameters
BUFFER_SIZE = int(1e5)  # Replay buffer size
BATCH_SIZE = 64         # Minibatch size
GAMMA = 0.99            # Discount factor
EPS_START = 1.0         # Starting value of epsilon, for epsilon-greedy action selection
EPS_END = 0.01          # Minimum value of epsilon
EPS_DECAY = 0.995       # Epsilon decay rate
TAU = 1e-3              # For soft update of target parameters
LR = 5e-4               # Learning rate
NUM_EPISODES = 2000     # Maximum number of training episodes
MAX_T = 1000            # Maximum number of timesteps per episode
UPDATE_EVERY = 4        # How often to update the network

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For reproducibility
seed = 0
env.action_space.seed(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Example of creating the network (optional, for testing in the notebook)
# state_size = env.observation_space.shape[0]
# action_size = env.action_space.n
# q_network_example = QNetwork(state_size, action_size, seed).to(device)
# print(q_network_example)

In [ ]:
# Need to import namedtuple from collections
from collections import namedtuple

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size) # deque and random are already imported in the first cell
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [ ]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.

        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.

        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy()).astype(int)
        else:
            return random.choice(np.arange(self.action_size)).astype(int)

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets) # Mean Squared Error loss
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [ ]:
# Initialize Agent
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = Agent(state_size=state_size, action_size=action_size, seed=seed)

def dqn_train():
    """Main training loop for DQN agent."""
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = EPS_START                    # initialize epsilon
    for i_episode in range(1, NUM_EPISODES+1):
        state, info = env.reset() # Use env.reset(seed=seed+i_episode) for newer gym versions if needed for strict reproducibility across runs
        score = 0
        for t in range(MAX_T):
            action = agent.act(state, eps)
            next_state, reward, done, truncated, _ = env.step(action)
            agent.step(state, action, reward, next_state, done or truncated)
            state = next_state
            score += reward
            if done or truncated:
                break
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(EPS_END, EPS_DECAY*eps) # decrease epsilon
        print(f'\rEpisode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}', end="")
        if i_episode % 100 == 0:
            print(f'\rEpisode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}')
        if np.mean(scores_window)>=200.0:
            print(f'\nEnvironment solved in {i_episode-100:d} episodes!\tAverage Score: {np.mean(scores_window):.2f}')
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint_lunar_lander.pth')
            break
    return scores

scores = dqn_train()

# Plotting the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.title('Score per Episode')
plt.show()

In [ ]:
# Optional: Code to load a saved model and visualize its performance

# Initialize the environment with rendering
# Note: If you are running this in a headless environment (like a remote server or some notebooks),
# rendering might not work or might require virtual display setup (e.g., Xvfb).
# For local execution with a display, 'human' render mode should work.
try:
    env_render = gym.make('LunarLander-v3', render_mode='human')
    state_size = env_render.observation_space.shape[0]
    action_size = env_render.action_space.n

    # Load the trained agent
    agent_trained = Agent(state_size=state_size, action_size=action_size, seed=seed) # Use the same seed or a different one
    agent_trained.qnetwork_local.load_state_dict(torch.load('checkpoint_lunar_lander.pth', map_location=device))
    agent_trained.qnetwork_local.to(device) # Ensure model is on the correct device

    print("Visualizing trained agent performance for a few episodes...")
    for i in range(5):  # Visualize 5 episodes
        state, info = env_render.reset()
        score = 0
        done_or_truncated = False
        max_steps_render = 1000 # Max steps for a rendered episode
        for t in range(max_steps_render):
            action = agent_trained.act(state, eps=0.01) # Use a small epsilon for some exploration or 0 for pure exploitation
            env_render.render()
            next_state, reward, done, truncated, _ = env_render.step(action)
            state = next_state
            score += reward
            done_or_truncated = done or truncated
            if done_or_truncated:
                break
        print(f"Episode {i+1}: Score = {score}")
    env_render.close()
except Exception as e:
    print(f"Could not run visualization. This might be due to a headless environment or missing display. Error: {e}")
    print("Skipping visualization.")